# **OpenMP Tasking – Recursive Algorithm**

### **Section 1: Serial Recursive Fibonacci (Baseline)**

In [4]:
%%bash
cat <<EOF > fibonacci.c
#include <stdio.h>
#include <time.h>

long fib(int n){
if(n<=1)
return n;
return fib(n-1)+fib(n-2);
}

int main(){
int n=40;
clock_t start=clock();
long result=fib(n);
clock_t end=clock();

printf("Fibonacci(%d) = %ld\n",n,result);
printf("Execution Time = %f seconds\n",(double)(end-start)/CLOCKS_PER_SEC);
return 0;
}
EOF

gcc fibonacci.c -o fibonacci
./fibonacci

Fibonacci(40) = 102334155
Execution Time = 0.605486 seconds


### **Section 2: OpenMP Task-based Fibonacci (Basic Tasking)**

In [21]:
%%bash
cat <<EOF > fib_omp_task.c
#include <stdio.h>
#include <omp.h>

long fib(int n){
if(n<=1)
return n;

long x=0,y=0;

#pragma omp task shared(x)
x=fib(n-1);

#pragma omp task shared(y)
y=fib(n-2);

#pragma omp taskwait
return x+y;
}

int main(){
int n=40;
long result=0;
double start=omp_get_wtime();

#pragma omp parallel
{
#pragma omp single
result=fib(n);
}

double end=omp_get_wtime();
printf("Fibonacci(%d) = %ld\n",n,result);
printf("Execution Time = %f seconds\n",end-start);
return 0;
}
EOF

gcc -fopenmp fib_omp_task.c -o fib_omp_task
./fib_omp_task


Fibonacci(40) = 102334155
Execution Time = 49.375300 seconds


### **Section 3: Task Creation and Synchronization using taskwait**

In [11]:
%%bash
cat <<EOF > fib_omp_task_section3.c
#include <stdio.h>
#include <omp.h>

long long fib(int n){
if(n<=1)return n;
long long x,y;
#pragma omp task shared(x)
x=fib(n-1);
#pragma omp task shared(y)
y=fib(n-2);
#pragma omp taskwait
return x+y;
}

int main(){
int n=40;
long long result;
double start=omp_get_wtime();

#pragma omp parallel
{
#pragma omp single
result=fib(n);
}

double end=omp_get_wtime();
printf("Fib(%d)=%lld\n",n,result);
printf("Execution Time=%f seconds\n",end-start);
return 0;
}
EOF

gcc -fopenmp fib_omp_task_section3.c -o fib_omp_task_section3
./fib_omp_task_section3

Fib(40)=102334155
Execution Time=49.242446 seconds



### **Section 4: Performance Analysis of OpenMP Tasking**

In [18]:
%%bash
cat <<EOF > fib_omp_task_n35.c
#include <stdio.h>
#include <omp.h>

long fib(int n){
if(n<=1)
return n;

long x=0,y=0;

#pragma omp task shared(x)
x=fib(n-1);

#pragma omp task shared(y)
y=fib(n-2);

#pragma omp taskwait
return x+y;
}

int main(){
int n=35;
long result=0;
double start=omp_get_wtime();

#pragma omp parallel
{
#pragma omp single
result=fib(n);
}

double end=omp_get_wtime();
printf("Fibonacci(%d) = %ld\n",n,result);
printf("Execution Time = %f seconds\n",end-start);
return 0;
}

EOF

gcc -fopenmp fib_omp_task_n35.c -o fib_omp_task_n35
./fib_omp_task_n35

Fibonacci(35) = 9227465
Execution Time = 4.734168 seconds


### **Section 5: Recursive Divide-and-Conquer Sum using OpenMP Tasks**

In [22]:
%%bash
cat <<EOF > array_sum_omp_task.c
#include <stdio.h>
#include <omp.h>

int sum(int a[],int l,int r){
if(l==r)
return a[l];

int mid=(l+r)/2;
int x=0,y=0;

#pragma omp task shared(x)
x=sum(a,l,mid);

#pragma omp task shared(y)
y=sum(a,mid+1,r);

#pragma omp taskwait
return x+y;
}

int main(){
int a[]={1,2,3,4,5,6,7,8};
int n=8;
int result=0;

#pragma omp parallel
{
#pragma omp single
result=sum(a,0,n-1);
}

printf("Array Sum = %d\n",result);
return 0;
}
EOF

gcc -fopenmp array_sum_omp_task.c -o array_sum_omp_task
./array_sum_omp_task

Array Sum = 36
